In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/tropical_cyclones/src/code
%ls

import imp 
# import helper.py
helper = imp.new_module('helper_functions')
exec(open("./helper_functions.py").read(), helper.__dict__)
# import models.py
models = imp.new_module('models')
exec(open("./models.py").read(), models.__dict__)
# import data_processor.py
dp = imp.new_module('data_processor')
exec(open("./data_processor.py").read(), dp.__dict__)

In [ ]:
# insert your desired path to work on
%cd /content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/data
%ls

In [ ]:
# general imports
import random
import glob
import os
import sys
sys.stdout.flush()
import pandas as pd
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import imageio
import os.path
import time
from PIL import Image
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import ndimage
from google.colab.patches import cv2_imshow
import random
from shapely.geometry import Point
import re
import pickle
import scipy
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import datetime

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras import Input
from tensorflow.keras.applications import resnet50, mobilenet_v2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import concatenate, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, TruePositives, FalsePositives, TrueNegatives, FalseNegatives
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

np.set_printoptions(precision=4)

## Train on data according to csv split into train, val and test sets.

Prepare the tf.data.Dataset instances to be fed to the model.

In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_VV+VH+WS"
NETWORK = "ResNet"     # options: ["Mobile", "ResNet"]
MIN_HEIGHT = 700
MIN_WIDTH = 400
NUM_VARS = False
NORMALISE = True
ROTATE = False
BATCH_SIZE = 16  # creates batches of 16 elements
BUFFER_SIZE = 100
EPOCHS = 10
LEARNING_RATE = 0.0001
##################################

# load data
train_images, train_labels, train_bbox = helper.load_data("{}/csv/training.csv".format(main_dir), with_cat = False)
val_images, val_labels, val_bbox = helper.load_data("{}/csv/val.csv".format(main_dir), with_cat = False)
test_images, test_labels, test_bbox = helper.load_data("{}/csv/test.csv".format(main_dir), with_cat = False)


# create an instance of the DataProcessor
processor = dp.DataProcessor(model = NETWORK,
                             min_height = MIN_HEIGHT,
                             min_width = MIN_WIDTH,
                             normalise = NORMALISE,           # perform normalisation
                             rotate = ROTATE,                 # perform rotation
                             plot_light = False,              # plot only select_crop() images
                             plot_extensive = False,          # plot extensively all images
                             show_prints = False)


# generate datasets
train_dataset = helper.prepare_dataset(processor, train_images, train_labels, train_bbox)
val_dataset = helper.prepare_dataset(processor, val_images, val_labels, val_bbox)
test_dataset = helper.prepare_dataset(processor, test_images, test_labels, test_bbox)
#for image, label in train_dataset:
#  plt.imshow(image)
#  plt.show()
#  print("FINAL - image: {}, label: {}".format(image.shape, label))
#  print("FINAL - image: {}, max: {}, min: {}, label: {}".format(image.shape, np.max(image), np.min(image),  label))


# configure for performance
train_dataset = helper.configure_for_performance(train_dataset, BUFFER_SIZE, BATCH_SIZE, shuffle = True)
val_dataset = helper.configure_for_performance(val_dataset, BUFFER_SIZE, BATCH_SIZE)
test_dataset = helper.configure_for_performance(test_dataset, BUFFER_SIZE, BATCH_SIZE)

Script to perform end-to-end training in the model.

In [ ]:
# directory to save results
dir = '{}_Numeric-{}_BatchSize-{}_Epochs-{}_Norm-{}'.format(NETWORK, str(NUM_VARS)[0], str(BATCH_SIZE), str(EPOCHS), str(NORMALISE)[0])
save_dir = 'classification_results/identification/' + dir + '/'
os.makedirs(save_dir, exist_ok=True)

# MAKE MODEL
if NETWORK == "Mobile":
    model = models.make_MobileNet(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE, detection = True)
elif NETWORK == "ResNet":
    model = models.make_ResNet(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE, detection = True)
else:
  sys.exit("Incert valid network model. Options: Mobile or ResNet (case sensitive)")

# CREATE CALLBACKS
callbacks = [
    EarlyStopping(patience = 5, verbose = 1),
    ReduceLROnPlateau(factor = 0.1, patience = 5, min_lr = 0.00001, verbose = 1),
    ModelCheckpoint(save_dir + "best_model.h5", verbose = 1, save_best_only = True),
    TensorBoard(log_dir = save_dir + "logs/" + datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
]

# TRAIN THE NETWORK
H = model.fit(
    train_dataset,
    steps_per_epoch = len(train_dataset),
    validation_data = val_dataset,
    validation_steps = len(val_dataset),
    epochs = EPOCHS,
    callbacks = callbacks,
    verbose = 1,
    class_weight = {0:1.6, 1:1},
    shuffle = True)

In [ ]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir test/classification_results/ResNet_Numeric-F_BatchSize-16_Epochs-10_Norm-T/logs

In [ ]:
# EVALUATE MODEL
print("Loaded best weights of the training")
model.load_weights(save_dir + "best_model.h5")

results = model.evaluate(test_dataset, 
                         steps = len(test_dataset), 
                         verbose = 1)

In [ ]:
# MAKE PREDICTIONS
# Retrieve a batch of images from the test set
predictions = model.predict(test_dataset.)

predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n')
for label in test_labels_dataset:
  print(label)
#class_names = {0: "No eye", 1: "Eye"}
#plt.figure(figsize=(10, 10))
#for i in range(9):
#  ax = plt.subplot(3, 3, i + 1)
#  plt.imshow(image_batch[i].astype("uint8"))
#  plt.title(class_names[predictions[i]])
#  plt.axis("off")

## Train on data using the Stratified K Fold.



In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_WS+sWSO+cWSO"
NETWORK = "ResNet"     # options: ["Mobile", "ResNet"]
MIN_HEIGHT = 700
MIN_WIDTH = 400
NUM_VARS = False
NORMALISE = True
ROTATE = False
BATCH_SIZE = 8
BUFFER_SIZE = 100
EPOCHS = 20
FOLDS = 5
LEARNING_RATE = 0.0001
##################################

# STRATIFIED CROSS VALIDATION
def fold_cross_validation(main_dir, NETWORK, MIN_HEIGHT, MIN_WIDTH, NUM_VARS, NORMALISE, ROTATE, LEARNING_RATE, EPOCHS, BUFFER_SIZE, BATCH_SIZE, SPLIT_NUMBER):

  df = pd.read_csv(main_dir + "/csv/full_dataset.csv", converters={'bbox_shape': eval}).dropna()
  Y = df["label"]

  stratified_k_fold = StratifiedKFold(n_splits = SPLIT_NUMBER, random_state = 42, shuffle = False)

  VALIDATION_ACCURACY = []
  VALIDATION_LOSS = []
  VALIDATION_TP = []
  VALIDATION_FP = []
  VALIDATION_TN = []
  VALIDATION_FN = []
  VALIDATION_PRECISION = []
  VALIDATION_RECALL = []

  print("Entering in k fold cross validation")
  print("Dataset dim: {}".format(len(df)))

  # directory to save results
  dir = '{}_Numeric-{}_BatchSize-{}_{}x{}_lr-{}_Epochs-{}_Folds-{}_Norm-{}'.format(
      NETWORK, str(NUM_VARS)[0], BATCH_SIZE, MIN_HEIGHT, MIN_WIDTH, LEARNING_RATE, EPOCHS, SPLIT_NUMBER, str(NORMALISE)[0])
  save_dir = main_dir + '/classification_results/identification/' + dir + '/'
  fold_var = 1
  
  # create an instance of the DataProcessor
  processor = dp.DataProcessor(model = NETWORK,
                               min_height = MIN_HEIGHT,
                               min_width = MIN_WIDTH,
                               normalise = NORMALISE,           # perform normalisation
                               rotate = ROTATE,                 # perform rotation
                               plot_light = False,              # plot only select_crop() images
                               plot_extensive = False,          # plot extensively all images
                               show_prints = False)


  for train_index, val_index in stratified_k_fold.split(np.zeros(len(df)), Y):
    training_data = df.iloc[train_index]
    validation_data = df.iloc[val_index]

    train_images, train_labels, train_bbox = helper.load_data(df = training_data, with_cat = False)
    val_images, val_labels, val_bbox = helper.load_data(df = validation_data, with_cat = False)
    
    # generate datasets
    train_dataset = helper.prepare_dataset(processor, train_images, train_labels, train_bbox)
    val_dataset = helper.prepare_dataset(processor, val_images, val_labels, val_bbox)

    # configure for performance
    train_dataset = helper.configure_for_performance(train_dataset, BUFFER_SIZE, BATCH_SIZE, shuffle = True)
    val_dataset = helper.configure_for_performance(val_dataset, BUFFER_SIZE, BATCH_SIZE)

    # CREATE NEW MODEL
    """
    # multi GPU strategy
    strategy = tf.distribute.MirroredStrategy()
    print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
    with strategy.scope():
        model = make_resNet(IMG_SIZE, start_from_scratch=True)
    """
    if NETWORK == "Mobile":
        model = models.make_MobileNet(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE, detection = True)
    elif NETWORK == "ResNet":
        model = models.make_ResNet(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE, detection = True)
    else:
        sys.exit("Incert valid network model. Options: Mobile or ResNet (case sensitive)")

    # CREATE CALLBACKS
    callbacks = [
        EarlyStopping(patience = 10, verbose = 1),
        ReduceLROnPlateau(factor = 0.1, patience = 5, min_lr = 0.00001, verbose = 1),
        ModelCheckpoint(save_dir + helper.get_model_name(fold_var), verbose = 1, save_best_only = True),
        TensorBoard(log_dir = save_dir + "logs/" + datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
    ]

    # FIT THE MODEL
    history = model.fit(
        train_dataset,
        steps_per_epoch = len(train_dataset),
        validation_data = val_dataset,
        validation_steps = len(val_dataset), 
        epochs = EPOCHS,
        callbacks = callbacks,
        verbose = 1, 
        class_weight = {0:1.6, 1:1},
        shuffle = True)
    
    # PLOT HISTORY
    #print(history)


    # PLOT TRAIN/VALIDATION LOSSES
    fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 1, figsize=(10, 10))
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.grid(True)
    ax1.legend(loc='lower right')
    ax1.set(ylabel = "Accuracy",
            title = 'Training and Validation Accuracy')

    ax2.plot(history.history["loss"], label='Training Loss')
    ax2.plot(history.history["val_loss"], label='Validation Loss')
    ax2.plot(np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="r", label = "Best model")
    ax2.grid(True)
    ax2.legend(loc='upper right')
    ax2.set(xlabel = 'Epoch', 
          ylabel = 'Cross Entropy',
          title = 'Training and Validation Loss')
    plt.show()
    fig.savefig(save_dir + "model_" + str(fold_var) + ".jpg")
    

    ##################################################################################################
    time.sleep(12) # guarantees enough time so that weights are saved and can be loaded after

    # LOAD BEST MODEL
    print("Loaded best weights of the training")
    model.load_weights(save_dir + helper.get_model_name(fold_var))     # Maybe a problem with colab, weights saved and loaded too fast giving concurrency problems

    # EVALUATE PERFORMANCE of the model
    results = model.evaluate(val_dataset, 
                             steps = len(val_dataset), 
                             verbose = 1)

    results = dict(zip(model.metrics_names, results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    VALIDATION_TP.append(results["tp"])
    VALIDATION_FP.append(results["fp"])
    VALIDATION_TN.append(results["tn"])
    VALIDATION_FN.append(results["fn"])
    VALIDATION_PRECISION.append(results["precision"])
    VALIDATION_RECALL.append(results["recall"])

    # MAKE PREDICTIONS
    predictions = model.predict(val_dataset)
    predictions = tf.where(predictions < 0.5, 0, 1)

    conf_mat = confusion_matrix(val_labels, predictions, labels = [0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = [0,1])
    conf_mat_display = disp.plot()
    plt.savefig(save_dir + "confusion_matrix_" + str(fold_var) + ".jpg")

    tf.keras.backend.clear_session()

    fold_var += 1

    # save the values for each fold
    csv_dir = save_dir + 'csv/'
    os.makedirs(csv_dir, exist_ok=True)
    with open(csv_dir + 'test_accuracy.pkl', 'wb') as handle:
        pickle.dump(VALIDATION_ACCURACY, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(csv_dir + 'test_loss.pkl', 'wb') as handle:
        pickle.dump(VALIDATION_LOSS, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(csv_dir + 'test_tp.pkl', 'wb') as handle:
        pickle.dump(VALIDATION_TP, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(csv_dir + 'test_fp.pkl', 'wb') as handle:
        pickle.dump(VALIDATION_FP, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(csv_dir + 'test_tn.pkl', 'wb') as handle:
        pickle.dump(VALIDATION_TN, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(csv_dir + 'test_fn.pkl', 'wb') as handle:
        pickle.dump(VALIDATION_FN, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(csv_dir + 'test_precision.pkl', 'wb') as handle:
        pickle.dump(VALIDATION_PRECISION, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(csv_dir + 'test_recall.pkl', 'wb') as handle:
        pickle.dump(VALIDATION_RECALL, handle, protocol=pickle.HIGHEST_PROTOCOL)

  return

##################################################################################################
# test
fold_cross_validation(main_dir, NETWORK, MIN_HEIGHT, MIN_WIDTH, NUM_VARS, NORMALISE, ROTATE, LEARNING_RATE, EPOCHS, BUFFER_SIZE, BATCH_SIZE, FOLDS)